<table align="center" width=100%>
    <tr>
        <td width="20%">
            <img src="homework.png">
        </td>
        <td>
            <div align="center">
                <font color="#21618C" size=8px>
                    <b> Take-Home <br>(Day1 and Day 2)
                    </b>
                </font>
            </div>
        </td>
        <td width="25%">
            <img src="GL-2.png">
        </td>
    </tr>
</table>

#### Import the required libraries

In [1]:
# import 'Pandas' 
import pandas as pd 

# import 'Numpy' 
import numpy as np

# import subpackage of Matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# import 'Seaborn' 
import seaborn as sns

# to suppress warnings 
from warnings import filterwarnings
filterwarnings('ignore')

# display all columns of the dataframe
pd.options.display.max_columns = None

# display all rows of the dataframe
pd.options.display.max_rows = None
 
# to display the float values upto 6 decimal places     
pd.options.display.float_format = '{:.6f}'.format

# import train-test split 
from sklearn.model_selection import train_test_split

# import various functions from statsmodels
import statsmodels
import statsmodels.api as sm

# import StandardScaler to perform scaling
from sklearn.preprocessing import StandardScaler 

# import various functions from sklearn 
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score

# import function to perform feature selection
from sklearn.feature_selection import RFE
# set the plot size using 'rcParams'
# once the plot size is set using 'rcParams', it sets the size of all the forthcoming plots in the file
# pass width and height in inches to 'figure.figsize' 
plt.rcParams['figure.figsize'] = [15,8]

#### Read the data

Load the csv file and print the first five observations.

In [2]:
data = pd.read_csv('bank_churn.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'bank_churn.csv'

Our objective is to predict whether the customer has closed the bank account or not.

**The data definition is as follows:** <br>

**CreditScore**: Credit score of the customer 

**Geography**: Resident country of the customer

**Gender**: Gender of the customer

**Age**: Age of the customer

**NumOfYrsWithBank**: Years for which the customer has been with the bank

**Balance**: Bank balance of the customer in Euro

**NumOfProducts**: Number of bank facilities for which customer has opted

**HasCrCard**: Whether the customer has credit card or not (1 = Yes, 0 = No)

**Closed_Acc**: Whether the customer has closed the bank account or not (1 = Yes, 0 = No) (target/dependent variable)

### Let's begin with some hands-on practice exercises

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>1. Is the target variable imbalanced? </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
data['Closed_Acc'].value_counts()

In [ ]:
#Yeah the target variable is imbalanced

In [ ]:
data.isnull().sum()

In [ ]:
#The data has no null values

In [ ]:
data.info()

In [ ]:
data['Geography'].value_counts()

In [ ]:
data['Gender'].value_counts()

In [ ]:
df = pd.get_dummies(data,drop_first=True)

In [ ]:
df.head()

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>2. Build a full logistic model and calculate the odds for each variable (consider the probability  threshold as 0.25)</b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
X = df.drop('Closed_Acc',axis=1)
y = df['Closed_Acc']
newx = sm.add_constant(X)

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(newx,y,train_size=0.80,random_state=1)

In [ ]:
logreg = sm.Logit(ytrain,xtrain).fit()

In [ ]:
logreg.summary()

In [ ]:
df_odds = pd.DataFrame(np.exp(logreg.params),columns=['Odds'])

In [ ]:
df_odds

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>3. Calculate the Specificity and Sensitivity from the confusion matrix of the full model (consider the probability threshold as 0.25)
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
ypred_prob = logreg.predict(xtest)
ypred = [0 if x < 0.25 else 1 for x in ypred_prob]

In [ ]:
cm = confusion_matrix(ytest,ypred)

In [ ]:
cm

In [ ]:
TN = cm[0,0]
FP = cm[0,1]
FN = cm[1,0]
TP = cm[1,1]

In [ ]:
#Specificity(TNR)-Recall for class 0

In [ ]:
(TN/(TN+FP))

In [ ]:
#Sensitivity(TPR)-Recall for class 1

In [ ]:
(TP/(FN+TP))

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>4. Build a logistic model on the 6 features obtained by RFE and plot the ROC curve 
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
X = df.drop('Closed_Acc',axis=1)
y = df['Closed_Acc']
xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=0.80,random_state=1)

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
rfe = RFE(estimator=lr,n_features_to_select=6)
rfe.fit(xtrain,ytrain)

In [ ]:
feature_index = pd.Series(rfe.ranking_,index=xtrain.columns).sort_values(ascending=False)
impfeature_rfe = feature_index[feature_index==1].index

In [ ]:
impfeature_rfe

In [ ]:
X = df[['Age','NumOfProducts','HasCrCard','Geography_Germany','Geography_Spain','Gender_Male']]
y = df['Closed_Acc']

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=0.80,random_state=1)

In [ ]:
lr =  LogisticRegression()

In [ ]:
lr.fit(xtrain,ytrain)

In [ ]:
ypred_lr = lr.predict(xtest)

In [ ]:
ypred_prob_lr = lr.predict_proba(xtest)

In [ ]:
from sklearn.metrics import roc_curve,roc_auc_score

In [ ]:
fpr,tpr,threshold = roc_curve(ytest,ypred_prob_lr[:,1])

In [ ]:
plt.plot(fpr,tpr)
plt.xlabel('FPR(1-specificity)')
plt.ylabel('TPR(sensitivity)')
plt.plot([0,1],[0,1],'r--')
print(roc_auc_score(ytest,ypred_prob_lr[:,1]))
plt.show()

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>5. Obtain the optimal value of cut-off probability for the full model using the Youden's index 
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
X = df.drop('Closed_Acc',axis=1)
y = df['Closed_Acc']
xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=0.80,random_state=1)

In [ ]:
lr =  LogisticRegression()
lr.fit(xtrain,ytrain)
ypred_lr = lr.predict(xtest)
ypred_prob_lr = lr.predict_proba(xtest)
fpr,tpr,threshold = roc_curve(ytest,ypred_prob_lr[:,1])

In [ ]:
youden_table = pd.DataFrame({'TPR':tpr,'FPR':fpr,'Threshold':threshold})

In [ ]:
youden_table.head()

In [ ]:
youden_table['difference'] = youden_table['TPR']-youden_table['FPR']
youden_table = youden_table.sort_values('difference',ascending=False)

In [ ]:
youden_table.head(1)

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>6. Consider the cut-off probability obtained from Youden's index for the full model and calculate the following measures:<br>
                    a. f-1 score <br>
                    b. Accuracy<br>
                    c. Kappa Score
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
X = df.drop('Closed_Acc',axis=1)
y = df['Closed_Acc']
newx = sm.add_constant(X)
xtrain,xtest,ytrain,ytest = train_test_split(newx,y,train_size=0.80,random_state=1)

In [ ]:
logreg = sm.Logit(ytrain,xtrain).fit()

In [ ]:
ypred_prob=logreg.predict(xtest)

In [ ]:
ypred = [0 if x < 0.204008 else 1 for x in ypred_prob]

In [ ]:
print(classification_report(ytest,ypred))

In [ ]:
print(cohen_kappa_score(ytest,ypred))

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>7. Identify the variables involved in multicollinearity 
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
X = df.drop('Closed_Acc',axis=1)
y = df['Closed_Acc']

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_list = []
for i in range(X.shape[1]):
    vif_list.append(variance_inflation_factor(X.values,i))
pd.DataFrame({'Features':X.columns,'VIF':vif_list}).sort_values(by='VIF',ascending=False)

In [ ]:
#Credit_score,Age,NumOfProducts are involved in multicollinearity

In [ ]:
X = df.drop(['Closed_Acc','CreditScore'],axis=1)
y = df['Closed_Acc']

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif_list = []
for i in range(X.shape[1]):
    vif_list.append(variance_inflation_factor(X.values,i))
pd.DataFrame({'Features':X.columns,'VIF':vif_list}).sort_values(by='VIF',ascending=False)

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>8. Build a logistic regression model using the categorical variables and the variables obtained after calculating VIF. Also, plot the ROC curve and compute the AUC score (consider the cut-off probability as 0.6)
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
lr =  LogisticRegression()
lr.fit(xtrain,ytrain)
ypred_lr = lr.predict(xtest)
ypred_prob_lr = lr.predict_proba(xtest)
fpr,tpr,threshold = roc_curve(ytest,ypred_prob_lr[:,1])

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>9. Consider the costs of false negatives and false positives as 2 and 0.5 respectively to obtain the optimal cut-off probability for which the total cost will be minimum
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
# type your code here

<table align="left">
    <tr>
        <td width="6%">
            <img src="question_icon.png">
        </td>
        <td>
            <div align="left", style="font-size:120%">
                <font color="#21618C">
                    <b>10. Build a full logistic model using the optimal cut-off probability obtained in Q9. Also, plot the confusion matrix and ROC curve along with the AUC score  
                    </b>
                </font>
            </div>
        </td>
    </tr>
</table>

In [ ]:
# type your code here